# Practice 5 - Parameter Efficient Tunning (ALBERT, DistillBERT) & Prompt Tuning

### Dataset Load & Preprocessing

In [31]:
with open('/kaggle/input/2024-1-nlp-5/Korean_movie_reviews_2016.txt/Korean_movie_reviews_2016.txt', encoding='utf-8') as f:
    docs = [doc.strip().split('\t') for doc in f]
    docs = [(doc[0], int(doc[1])) for doc in docs if len(doc) == 2]
    texts, labels = zip(*docs)
    
words_list = [doc.strip().split() for doc in texts]
print(words_list[:2])
print(len(texts))

[['부산', '행', '때문', '너무', '기대하고', '봤'], ['한국', '좀비', '영화', '어색하지', '않게', '만들어졌', '놀랍']]
165384


In [32]:
from tensorflow.keras.utils import to_categorical
y_one_hot = to_categorical(labels)

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(texts, y_one_hot, test_size=0.2, random_state=0)

## 실습 5.1 - Parameter Efficient Tuning with ALBERT

### Load Tokenizer & Dataset Split

In [34]:
from transformers import BertTokenizer, TFAlbertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained("kykim/albert-kor-base")

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'AlbertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


In [35]:
X_train_tokenized = tokenizer(X_train, return_tensors="np", max_length=30, padding='max_length', truncation=True)
X_test_tokenized = tokenizer(X_test, return_tensors="np", max_length=30, padding='max_length', truncation=True)

### Load Model

In [36]:
albert_model = TFAlbertForSequenceClassification.from_pretrained("kykim/albert-kor-base", num_labels=2, from_pt=True)
albert_model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertForSequenceClassification: ['albert.embeddings.position_ids', 'sop_classifier.classifier.weight', 'sop_classifier.classifier.bias']
- This IS expected if you are initializing TFAlbertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFAlbertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Model: "tf_albert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 albert (TFAlbertMainLayer)  multiple                  13186816  
                                                                 
 dropout_29 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
Total params: 13188354 (50.31 MB)
Trainable params: 13188354 (50.31 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## 실습 및 과제 5.1

Finetune your ALBERT model with "Korean_movie_reviews_2016.txt" 데이터셋

In [37]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy

train_dataset = tf.data.Dataset.from_tensor_slices((dict(X_train_tokenized), y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(X_test_tokenized), y_test))

train_dataset = train_dataset.shuffle(len(X_train)).batch(32)
test_dataset = test_dataset.batch(32)

loss = CategoricalCrossentropy(from_logits=True)
albert_model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

history = albert_model.fit(train_dataset, validation_data=test_dataset,epochs=1)

4135/4135 [==============================] - 653s 150ms/step - loss: 0.7119 - accuracy: 0.5112 - val_loss: 0.6992 - val_accuracy: 0.5263


In [38]:
import numpy as np
albert_model.evaluate(dict(X_test_tokenized), np.array(y_test))

1034/1034 [==============================] - 70s 57ms/step - loss: 0.6992 - accuracy: 0.5263


[0.6991918683052063, 0.5262871384620667]

In [39]:
y_preds = albert_model.predict(dict(X_test_tokenized))
prediction_probs = tf.nn.softmax(y_preds.logits,axis=1).numpy()
y_predictions = np.argmax(prediction_probs, axis=1)
y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_predictions, y_test))

1034/1034 [==============================] - 67s 56ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.53      0.69     33077

    accuracy                           0.53     33077
   macro avg       0.50      0.26      0.34     33077
weighted avg       1.00      0.53      0.69     33077



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 실습 5.2 - Parameter Efficient Tuning with DistillBERT

In [41]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("monologg/distilkobert")

X_train_tokenized_DistillBERT = DistillBERT_tokenizer(X_train, return_tensors="np", max_length=30, padding='max_length', truncation=True)
X_test_tokenize_DistillBERT = DistillBERT_tokenizer(X_test, return_tensors="np", max_length=30, padding='max_length', truncation=True)

### Load Model & Training

In [42]:
model_distillBERT = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2, from_pt=True)
model_distillBERT.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'cla

Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMa  multiple                  66362880  
 inLayer)                                                        
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_49 (Dropout)        multiple                  0         
                                                                 
Total params: 66955010 (255.41 MB)
Trainable params: 66955010 (255.41 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


## 실습 및 과제 5.2 

Finetune your DistilKoBERT model with "Korean_movie_reviews_2016.txt" 데이터셋

In [43]:
distill_train_dataset = tf.data.Dataset.from_tensor_slices((dict(X_train_tokenized_DistillBERT), y_train))
distill_test_dataset = tf.data.Dataset.from_tensor_slices((dict(X_test_tokenize_DistillBERT), y_test))

distill_train_dataset = distill_train_dataset.shuffle(len(X_train)).batch(32)
distill_test_dataset = distill_test_dataset.batch(32)

loss = CategoricalCrossentropy(from_logits=True)
model_distillBERT.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

history_distill = model_distillBERT.fit(distill_train_dataset, validation_data=distill_test_dataset, epochs=1)

4135/4135 [==============================] - 379s 84ms/step - loss: 0.6925 - accuracy: 0.5232 - val_loss: 0.6918 - val_accuracy: 0.5263


## Evaluation

In [44]:
model_distillBERT.evaluate(dict(X_test_tokenized), np.array(y_test))

1034/1034 [==============================] - 35s 29ms/step - loss: 0.6918 - accuracy: 0.5263


[0.6917791366577148, 0.5262871384620667]

In [46]:
y_preds = model_distillBERT.predict(dict(X_test_tokenized))
prediction_probs = tf.nn.softmax(y_preds.logits,axis=1).numpy()
y_predictions = np.argmax(prediction_probs, axis=1)
y_test = np.argmax(y_test, axis=1)
from sklearn.metrics import classification_report
print(classification_report(y_predictions, y_test))

1034/1034 [==============================] - 33s 28ms/step
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.53      0.69     33077

    accuracy                           0.53     33077
   macro avg       0.50      0.26      0.34     33077
weighted avg       1.00      0.53      0.69     33077



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 실습 5.3

- BERT 모델, ALBERT 모델, DistillBERT 모델의 크기와 성능을 비교하시오.

## 실습 5.4 - Prompt Engineering

### 5.4.1 - 프롬프트 작성 원칙
모델이 최대한 정확하고 유용한 정보를 제공할 수 있도록 효과적인 프롬프트를 작성하는 것이 매우 중요합니다. 좋은 프롬프트를 만들기 위해서 다음과 같은 원칙을 고려합니다.

1. 명확성과 구체성
질문은 명확하고 구체적이어야 합니다. 모호한 질문은 LLM 모델의 혼란을 초래할 수 있기 때문입니다.
예시: "다음 주 주식 시장에 영향을 줄 수 있는 예정된 이벤트들은 무엇일까요?"는 "주식 시장에 대해 알려주세요."보다 더 구체적이고 명확한 질문입니다.
2. 배경 정보를 포함
모델이 문맥을 이해할 수 있도록 필요한 배경 정보를 제공하는 것이 좋습니다. 이는 환각 현상(hallucination)이 발생할 위험을 낮추고, 관련성 높은 응답을 생성하는 데 도움을 줍니다.
예시: "2020년 미국 대선의 결과를 바탕으로 현재 정치 상황에 대한 분석을 해주세요."
3. 간결함
핵심 정보에 초점을 맞추고, 불필요한 정보는 배제합니다. 프롬프트가 길어지면 모델이 덜 중요한 부분에 집중하거나 상당한 영향을 받는 문제가 발생할 수 있습니다.
예시: "2021년에 발표된 삼성전자의 ESG 보고서를 요약해주세요."
4. 열린 질문 사용
열린 질문을 통해 모델이 자세하고 풍부한 답변을 제공하도록 유도합니다. 단순한 '예' 또는 '아니오'로 대답할 수 있는 질문보다는 더 많은 정보를 제공하는 질문이 좋습니다.
예시: "신재생에너지에 대한 최신 연구 동향은 무엇인가요?"
5. 명확한 목표 설정
얻고자 하는 정보나 결과의 유형을 정확하게 정의합니다. 이는 모델이 명확한 지침에 따라 응답을 생성하도록 돕습니다.
예시: "AI 윤리에 대한 문제점과 해결 방안을 요약하여 설명해주세요."
6. 언어와 문체
대화의 맥락에 적합한 언어와 문체를 선택합니다. 이는 모델이 상황에 맞는 표현을 선택하는데 도움이 됩니다.
예시: 공식적인 보고서를 요청하는 경우, "XX 보고서에 대한 전문적인 요약을 부탁드립니다."와 같이 정중한 문체를 사용합니다.

### 예시: 제품 리뷰 요약
* 지시: "아래 제공된 제품 리뷰를 요약해주세요."
* 예시: "예를 들어, '이 제품은 매우 사용하기 편리하며 배터리 수명이 길다'라는 리뷰는 '사용 편리성과 긴 배터리 수명이 특징'으로 요약할 수 있습니다."
* 맥락: "리뷰는 스마트워치에 대한 것이며, 사용자 경험에 초점을 맞추고 있습니다."
* 질문: "이 리뷰를 바탕으로 스마트워치의 주요 장점을 두세 문장으로 요약해주세요."

### 5.4.2 openai langchain 시스템 이용하기

In [47]:
# openai langchain 시스템 이용하기
!pip install langchain_core langchain_openai

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.1/127.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.4/327.4 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: orjson
    Found existing installation: orjson 3.9.10
    Uninstalling orjson-3.9.10:
      Successfully uninstalled orjson-3.9.10
ERROR: pip's dependency resolver does not curren

### 5.4.3 템플릿 만들기

In [48]:
#문자열 템플릿 - 다음 예제는 langchain_core.prompts 모듈의 PromptTemplate 클래스를 사용하여, 'name'과 'age'라는 두 개의 변수를 포함하는 프롬프트 템플릿을 정의하고 있습니다. 
#이 템플릿을 이용하여 실제 입력값을 해당 위치에 채워 넣어 완성된 프롬프트를 생성하는 과정을 보여줍니다.

from langchain_core.prompts import PromptTemplate

# 'name'과 'age'라는 두 개의 변수를 사용하는 프롬프트 템플릿을 정의
template_text = "안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다."

# PromptTemplate 인스턴스를 생성
prompt_template = PromptTemplate.from_template(template_text)

# 템플릿에 값을 채워서 프롬프트를 완성
filled_prompt = prompt_template.format(name="홍길동", age=30)

filled_prompt

'안녕하세요, 제 이름은 홍길동이고, 나이는 30살입니다.'

In [49]:
# 문자열 템플릿 결합 (PromptTemplate + PromptTemplate + 문자열)
combined_prompt = (
              prompt_template
              + PromptTemplate.from_template("\n\n아버지를 아버지라 부를 수 없습니다.")
              + "\n\n{language}로 번역해주세요."
)

combined_prompt

PromptTemplate(input_variables=['age', 'language', 'name'], template='안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다.\n\n아버지를 아버지라 부를 수 없습니다.\n\n{language}로 번역해주세요.')

In [ ]:
combined_prompt.format(name="홍길동", age=30, language="영어")

### 5.4.4 ChatOpenAI 인스턴스 이용하기

In [50]:
# ChatOpenAI 인스턴스를 생성하여 프롬프트 텍스트를 전달하고, 모델의 출력을 StrOutputParser를 통해 문자열로 변환하는 LLM 체인을 구성합니다.
# invoke 메소드를 사용하여 파이프라인을 실행하고, 최종적으로 문자열 출력을 얻습니다. 모델의 응답은 프롬프트에 주어진 문장을 영어로 번역한 텍스트가 출력됩니다.

from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model="gpt-3.5-turbo-0125",
                 temperature=0,  # 창의성 (0.0 ~ 2.0)
                 max_tokens=2048,  # 최대 토큰수
                 
                 # 본 토큰은 2024학년도 1학기 텍스트마이닝/자연어처리 수업의 과제 5를 위해서만 사용이 가능합니다.
                 # 본 API는 6월 종강 시 까지 유지될 예정 입니다.
                 # 본 API를 사용하는 모든 책임은 본인에게 있습니다.
                 openai_api_key="sk-proj-cipaHcsyXxD93NswHF64T3BlbkFJdZaufRzviOKm6fiY4OJc")
                

chain = combined_prompt | llm | StrOutputParser()
chain.invoke({"age":30, "language":"영어", "name":"홍길동"})

'Hello, my name is Hong Gil-dong and I am 30 years old.\n\nI cannot call my father "father."'

### 5.4.5 튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

In [51]:
# 2-튜플 형태의 메시지 목록으로 프롬프트 생성 (type, content)

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 대학교의 수업에 대한 내용을 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

chain = chat_prompt | llm

chain.invoke({"user_input": "세종대학교의 텍스트마이닝 수업에 대해 알려줘."})

AIMessage(content='세종대학교의 텍스트마이닝 수업은 자연어 처리와 통계학을 기반으로 한 텍스트 데이터 분석 기술을 다루는 수업입니다. 이 수업에서는 텍스트 데이터를 수집, 전처리하고, 텍스트 마이닝 기법을 활용하여 정보를 추출하고 분석하는 방법을 학습합니다. 또한, 텍스트 분류, 토픽 모델링, 감성 분석 등 다양한 텍스트 마이닝 기술을 실습을 통해 익히게 됩니다. 이 수업을 통해 학생들은 실제 텍스트 데이터를 다루며 데이터 분석 능력을 향상시킬 수 있습니다.', response_metadata={'token_usage': {'completion_tokens': 229, 'prompt_tokens': 73, 'total_tokens': 302}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a905d23f-0497-4e76-b9b9-12dab5155630-0', usage_metadata={'input_tokens': 73, 'output_tokens': 229, 'total_tokens': 302})

In [52]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 여행 전문가입니다."),
    ("user", "{user_input}"),
])

chain = chat_prompt | llm
chain.invoke({"user_input": "안녕하세요? 한국의 대표적인 관광지 3군데를 추천해주세요."})

AIMessage(content='안녕하세요! 한국의 대표적인 관광지 3군데를 추천해드리겠습니다.\n\n1. 경복궁 (Gyeongbokgung Palace): 서울에 위치한 경복궁은 조선 시대의 궁궐로, 한국의 역사와 전통을 경험할 수 있는 곳입니다. 아름다운 건물과 정원, 전통 복식을 입은 경비병들의 수문장 교대식 등을 관람할 수 있습니다.\n\n2. 부산 해운대해수욕장 (Haeundae Beach): 부산에 위치한 해운대해수욕장은 한국에서 가장 유명한 해변 중 하나로, 아름다운 백사장과 깨끗한 바다가 매력적입니다. 여름에는 해수욕을 즐기고, 주변 맛집과 상점을 즐기며 휴양을 즐길 수 있습니다.\n\n3. 경주 석굴암 (Seokguram Grotto): 경주에 위치한 석굴암은 석가모니불상이 모신 동굴로, 한국의 대표적인 유네스코 세계문화유산 중 하나입니다. 아름다운 동굴 내부의 불상과 경치는 꼭 한 번 방문해보길 추천합니다.\n\n이렇게 한국의 다양한 매력을 경험할 수 있는 관광지들을 추천해드렸습니다. 즐거운 여행 되세요!', response_metadata={'token_usage': {'completion_tokens': 437, 'prompt_tokens': 59, 'total_tokens': 496}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d50231c9-a18b-46fb-b72e-64d116a6224f-0', usage_metadata={'input_tokens': 59, 'output_tokens': 437, 'total_tokens': 496})

### 5.4.6 Model Paramter 설정

In [60]:
# 모델 파라미터 설정
params = {
    "temperature": 0.7,         # 생성된 텍스트의 다양성 조정
    "max_tokens": 100,          # 생성할 최대 토큰 수    
}

kwargs = {
    "frequency_penalty": 0.5,   # 이미 등장한 단어의 재등장 확률
    "presence_penalty": 0.5,    # 새로운 단어의 도입을 장려
}

# 모델 인스턴스를 생성할 때 파라미터 설정
model = ChatOpenAI(model="gpt-3.5-turbo-0125", 
                   
                   # 본 토큰은 2024학년도 1학기 텍스트마이닝/자연어처리 수업의 과제 5를 위해서만 사용이 가능합니다.
                   # 본 API는 6월 종강 시 까지 유지될 예정 입니다.
                   # 본 API를 사용하는 모든 책임은 본인에게 있습니다.
                   openai_api_key="sk-proj-cipaHcsyXxD93NswHF64T3BlbkFJdZaufRzviOKm6fiY4OJc",
                   
                   # user-defined hyperparamters
                   **params, model_kwargs = kwargs, stop = 100)


# 모델 호출
question = "태양계에서 가장 큰 행성은 무엇인가요?"
response = model.invoke(input=question)

# 전체 응답 출력
print(response)
print()
print(response.content)

content='가장 큰 행성은 목성입니다. 목성은 태양계에서 가장 많은 물질을 포함하고 있으며, 질량과 부피 모두에서 다른 행성들을 압도합니다.' response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 29, 'total_tokens': 98}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-3619eaae-1cd5-429d-a86a-47320a5a4cc3-0' usage_metadata={'input_tokens': 29, 'output_tokens': 69, 'total_tokens': 98}

가장 큰 행성은 목성입니다. 목성은 태양계에서 가장 많은 물질을 포함하고 있으며, 질량과 부피 모두에서 다른 행성들을 압도합니다.


## 실습 및 과제 5.4 

나만의 프롬프트를 이용하여 본인이 원하는 분야의 특정 시스템을 정의(1)하고 파라미터 값을 조정하고 해당 시스템에 대한 예시 질문을 5개 정도 만들어 langchain 모델을 최적화 하라.

In [67]:
# Define prompt template for movie recommendation system
movie_prompt = ChatPromptTemplate.from_messages([
    ("system", "안녕하세요! 영화 추천 시스템입니다."),
    ("user", "어떤 영화를 좋아하세요?"),
    ("system", "{user_favorite_genre} 장르 영화를 좋아하시는군요! 어떤 분위기의 영화를 찾으세요?"),
    ("user", "{user_preferred_mood} 분위기의 영화를 찾아요."),
    ("system", "알겠습니다! {user_favorite_genre} 장르 {user_preferred_mood} 분위기의 영화를 추천해 드릴게요."),
])

# Set model parameters
params = {
    "temperature": 0.8,
    "max_tokens": 200,
}

# Define example inputs
example_inputs = [
    "SF 영화 중에서 액션 영화를 좋아합니다. 긴장감 넘치는 분위기의 영화를 추천해주세요.",
    "코미디 영화를 좋아합니다. 가벼운 분위기의 영화를 추천해주세요.",
    "멜로 영화를 좋아합니다. 감동적인 분위기의 영화를 추천해주세요.",
    "스릴러 영화를 좋아합니다. 짜릿한 분위기의 영화를 추천해주세요.",
    "액션 영화를 좋아합니다. 최근에 나온 영화 중에서 추천해주세요.",
]

# Create language model instance
model = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    openai_api_key="sk-proj-cipaHcsyXxD93NswHF64T3BlbkFJdZaufRzviOKm6fiY4OJc",
    **params,
    stop="\n",  # Pass 'stop' parameter directly to ChatOpenAI constructor
)

# Generate model responses
for user_input in example_inputs:
    response = chain.invoke(user_input)  # Pass user_input directly
    print(f"Input: {user_input}\nResponse: {response.content}\n")


Input: SF 영화 중에서 액션 영화를 좋아합니다. 긴장감 넘치는 분위기의 영화를 추천해주세요.
Response: 제가 추천하는 SF 액션 영화는 "인터스텔라(Interstellar)"입니다. 이 영화는 우주 여행을 다루면서도 긴장감 넘치는 스토리와 시각적 효과로 관객을 매료시킵니다. 감독 크리스토퍼 놀란의 작품으로, 과학적인 요소와 감동적인 이야기가 조화를 이루는 작품입니다. 인터스텔라를 감상하면서 새로운 우주 여행의 모험을 경험해보세요.

Input: 코미디 영화를 좋아합니다. 가벼운 분위기의 영화를 추천해주세요.
Response: 제가 추천하는 가벼운 분위기의 코미디 영화는 "라라랜드"입니다. 이 영화는 로맨스와 음악을 테마로 한 유쾌한 이야기로, 매력적인 음악과 아름다운 영상이 함께 어우러져 있습니다. 코믹한 장면과 유머러스한 대사들이 풍부하게 담겨 있어 즐거운 시간을 보낼 수 있을 것입니다. 즐겁고 유쾌한 영화 감상을 기대해보세요!

Input: 멜로 영화를 좋아합니다. 감동적인 분위기의 영화를 추천해주세요.
Response: 제가 추천하는 감동적인 멜로 영화는 "라라랜드"입니다. 이 영화는 음악과 로맨스가 어우러진 멋진 이야기로 많은 이들의 마음을 사로잡았습니다. 또한 "노트북"이나 "타이타닉"도 매우 감동적인 멜로 영화 중 하나로 손꼽힙니다. 이 영화들을 감상하면 여러 감정을 경험할 수 있을 것입니다.

Input: 스릴러 영화를 좋아합니다. 짜릿한 분위기의 영화를 추천해주세요.
Response: 제가 추천하는 몇 가지 짜릿한 분위기의 스릴러 영화는 다음과 같습니다:

1. "곡성" (The Wailing, 2016) - 한국 영화로, 미스터리한 분위기와 긴장감 넘치는 스토리가 특징입니다.
2. "게임" (The Game, 1997) - 데이빗 핀처 감독의 작품으로, 예측 불가능한 전개와 긴장감 있는 연출이 인상적입니다.
3. "세븐" (Se7en, 1995) - 브래드 피트와 모건 프리먼 주연의 이 영화는 다크하고 끔찍한 분위기를 자아냅니다.